# Neural ODE

System identification on a possibly sparse dataset of autonomous ODEs

In [ ]:
import time
from tqdm import tqdm
import diffrax
import equinox as eqx  # https://github.com/patrick-kidger/equinox
from flax import nnx
import jax
import jax.nn as jnn
import jax.numpy as jnp
import jax.random as jr
import matplotlib.pyplot as plt
import optax  # https://github.com/deepmind/optax

In [ ]:
%matplotlib widget

In [ ]:
# data
nx = 2
dataset_size = 2_000 # number of sequences in the dataset
seq_len = 40 # length of each sequence (number of time steps, sampling time is irregular)

# model
width_size=64
depth=2

# optimization
batch_size = 32
lr = 1e-3
steps = 10_000

# logging
print_every = 100

In [ ]:
seed = 1234
key = jr.PRNGKey(seed)
ykey, tkey1, tkey2, model_key, loader_key = jr.split(key, 5)

In [ ]:
# generate 40 irregularly-spaced time series from the Lotka-Volterra model

y0 = 8*jr.uniform(ykey, (dataset_size, 2)) + 6

t0 = 0
t1 = 140 + jr.uniform(tkey1, (dataset_size,))
ts = jr.uniform(tkey2, (dataset_size, seq_len)) * (t1[:, None] - t0) + t0
ts = jnp.sort(ts)
dt0 = 0.1
params = jnp.array([0.1, 0.02, 0.4, 0.02]) 

def vector_field(t, y, args):
    prey, predator = y    
    α, β, γ, δ = args
    d_prey = α * prey - β * prey * predator
    d_predator = -γ * predator + δ * prey * predator
    d_y = jnp.array([d_prey, d_predator])
    return d_y

def solve(ts, y0):
    sol = diffrax.diffeqsolve(
        diffrax.ODETerm(vector_field),
        diffrax.Tsit5(),
        ts[0],
        ts[-1],
        dt0,
        y0,
        saveat=diffrax.SaveAt(ts=ts),
        args=params
    )
    return sol.ys

ys = jax.vmap(solve)(ts, y0)

ts.shape, ys.shape

In [ ]:
# normalize data

#mu = jnp.mean(jnp.mean(ys, axis=1), axis=0)#,
#std = jnp.mean(jnp.std(ys, axis=1), axis=0)
#ys = (ys - mu)/std

In [ ]:
# plot some data
plt.figure()
plt.plot(ts[:4, :].T, ys[:4, :, 0].T, "r--*")
plt.plot(ts[:4, :].T, ys[:4, :, 1].T, "b--*");

In [ ]:
class Func(nnx.Module):

    def __init__(self, nx, rngs:nnx.Rngs):
        self.linear = nnx.Linear(nx, nx, rngs=rngs)

    def __call__(self, t, x, args):
        dx = nnx.gelu(self.linear(x))
        return dx
    
class NeuralOde(nnx.Module):
    def __init__(self, func: Func,  **kwargs):
        super().__init__(**kwargs)
        self.func = func
    
    def __call__(self, ts, y0):
        solution = diffrax.diffeqsolve(
            diffrax.ODETerm(self.func),
            diffrax.Tsit5(),
            t0=ts[0],
            t1=ts[-1],
            dt0=ts[1] - ts[0],
            y0=y0,
            stepsize_controller=diffrax.PIDController(rtol=1e-3, atol=1e-6),
            saveat=diffrax.SaveAt(ts=ts),
        )
        return solution.ys

In [ ]:
vector_field_nn = Func(nx, rngs=nnx.Rngs(model_key)) # how to mix jax and nnx rngs?
nnx.display(vector_field_nn)

In [ ]:
x=jnp.ones((3, nx)) # (B, din)
y = vector_field_nn(_, jnp.ones((3, 2)), _)
y

In [ ]:
simulator = NeuralOde(vector_field_nn) # how to mix jax and nnx rngs?
nnx.display(simulator)

In [ ]:
# def simulate(ts, y0):
#     sol = diffrax.diffeqsolve(
#         diffrax.ODETerm(vector_field_nn),
#         diffrax.Tsit5(),
#         ts[0],
#         ts[-1],
#         dt0,
#         y0,
#         saveat=diffrax.SaveAt(ts=ts),
#         args=params
#     )
#     return sol.ys

ys_sim = simulator(ts[0], y0[0])

plt.figure()
plt.plot(ts[0], ys_sim[:, 0], "r*")

In [ ]:
#ys_sim_batch = jax.vmap(simulate)(ts, ys[:, 0, :])
batched_simulator = nnx.vmap(simulator)
batched_simulator(ts, ys[:, 0, :]).shape

In [ ]:
import optax

learning_rate = 0.005
momentum = 0.9

optimizer = nnx.Optimizer(vector_field_nn, optax.adamw(learning_rate, momentum))
metrics = nnx.MultiMetric(
  accuracy=nnx.metrics.Accuracy(), 
  loss=nnx.metrics.Average('loss'),
)

In [ ]:
def loss_fn(simulator):
  batched_simulator = nnx.vmap(simulator)
  ys_sim = batched_simulator(ts, ys[:, 0, :])
  return jnp.mean((ys - ys_sim)**2)

loss_fn(simulator)

In [ ]:
loss_grad_fn = nnx.jit(nnx.value_and_grad(loss_fn))
loss, grad = loss_grad_fn(simulator)